In [10]:
import sklearn
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_boston
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn import preprocessing
import random
from sklearn.ensemble import RandomForestRegressor
import cvxpy as cp
import matplotlib.pyplot as plt
from gurobipy import *
import multiprocessing as mp
import time
import gurobipy as gp
from gurobipy import GRB
import sklearn
import numpy as np
import mosek
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_boston
import pandas as pd
from sklearn import preprocessing
import random
from sklearn.ensemble import RandomForestRegressor
import cvxpy as cp
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
import multiprocessing as mp
import time
from sklearn.model_selection import train_test_split
import interpret
from interpret import *

from interpret.glassbox import ExplainableBoostingRegressor
from interpret.glassbox import ExplainableBoostingClassifier

pd.set_option('display.max_columns', None)

In [11]:
def plot_tradeoff_curve(test_acc,nonzero,color,ntrials,label1):
    results = pd.DataFrame()
    for i in range(0,len(test_acc)):
        results = results.append(pd.DataFrame(np.column_stack((test_acc[i],nonzero[i])),columns = ['test_acc','nonzero']))
    agg = results.groupby(['nonzero'], as_index=False).agg({'test_acc':['mean','std']})
    plt.figure(figsize=(16,10))
    plt.scatter(agg['nonzero'],agg['test_acc']['mean'],color = color, label = label1)
    plt.errorbar(agg['nonzero'],agg['test_acc']['mean'], 1.96*agg['test_acc']['std']/np.sqrt(ntrials),color = color)
    plt.xlabel('Number of Nonzero Features')
    plt.ylabel('Test Error')

In [12]:
def build_ebm(xTrain,yTrain,xTest,yTest,lambd,threshold,prob_type):
    
    
    if prob_type == 'Regression':
        ebm = ExplainableBoostingRegressor(n_jobs = 3).fit(xTrain,yTrain)
        pred = ebm.predict(xTest)
    if prob_type == 'Classification':
        ebm = ExplainableBoostingClassifier(n_jobs = 3).fit(xTrain,yTrain)
        pred = ebm.predict_proba(xTest)[:,1]
        
    ebm_local = ebm.explain_local(xTrain, yTrain)
    local_imp = []
    for i in range(0,len(yTrain)):
        scores = ebm_local._internal_obj.get('specific')[i].get('scores')
        local_imp.append(scores)
    local_imp = np.array(local_imp)
    test_imp = []
    ebm_test = ebm.explain_local(xTest,yTest)
    for i in range(0,len(yTest)):
        scores = ebm_test._internal_obj.get('specific')[i].get('scores')
        test_imp.append(scores)
    test_imp  = np.array(test_imp)
    return([local_imp,test_imp])

def solve_lasso(xTrain,yTrain,xTest,yTest,lambd,threshold,prob_type,ebm_result):
    local_imp = ebm_result[0]
    test_imp = ebm_result[1]
    all_features = pd.DataFrame(xTrain.columns,columns = ['Features'])
    w = cp.Variable(len(xTrain.columns),nonneg=True)
    if prob_type == 'Regression':
        objective = 0.5 * (1/len(yTrain))*cp.sum_squares(cp.matmul(local_imp,w)-yTrain) + \
                                                    lambd*cp.norm(w,1)
    if prob_type == 'Classification':
        loss = -cp.sum(cp.multiply(yTrain, local_imp@ w) - cp.logistic(local_imp @ w))
        objective =  (1/len(yTrain))*loss + lambd*cp.norm(w,1)
        

    prob = cp.Problem(cp.Minimize(objective))
    prob.solve(solver = cp.MOSEK ,mosek_params = {mosek.dparam.optimizer_max_time: 1000.0})
    weights = np.asarray(w.value)
    low_values_flags = np.abs(weights) < threshold  # Where values are low
    weights[low_values_flags] = 0 
    feature_ind = np.where(weights >0)[0]

    if len(feature_ind) == 0:
        if prob_type == 'Regression':
            train_error =  np.sqrt(np.mean((yTrain)**2))
            test_error = np.sqrt(np.mean((yTest)**2))
            nonzero_features = 0
            feature_importances = all_features
            feature_importances['Importances'] = 0
            return([feature_importances,test_error,nonzero_features,train_error])
        if prob_type == 'Classification':
            train_error =  .5
            test_error = .5
            nonzero_features = 0
            feature_importances = all_features
            feature_importances['Importances'] = 0
            return([feature_importances,test_error,nonzero_features,train_error])

    features_sub = xTrain.columns[feature_ind]
    train_pred_non_zero = np.array([np.array(row[feature_ind]) for row in local_imp])
    test_pred_non_zero = np.array([np.array(row[feature_ind]) for row in test_imp])

    if prob_type == 'Regression':
        final_tree_weights = cp.Variable(len(feature_ind), nonneg = True)
        objective_final = cp.sum_squares(cp.matmul(train_pred_non_zero,final_tree_weights)-yTrain) 
        prob = cp.Problem(cp.Minimize(objective_final))
        prob.solve(solver = cp.MOSEK ,mosek_params = {mosek.dparam.optimizer_max_time: 100.0})

        final_train_pred = train_pred_non_zero@final_tree_weights.value
        final_test_pred = test_pred_non_zero@final_tree_weights.value

        train_error =  np.sqrt(np.mean((yTrain -final_train_pred)**2))
        test_error = np.sqrt(np.mean((yTest -final_test_pred)**2))

        importances = np.sum(np.abs(test_pred_non_zero),axis = 0)
        feature_importances = pd.DataFrame(np.column_stack((features_sub,importances))\
                                          ,columns = ['Features','Importances'])
        feature_importances = pd.merge(all_features,feature_importances,on = 'Features',how = 'left').fillna(0).\
                                sort_values('Importances',ascending = False)
        nonzero_features = len(feature_ind)
        return([feature_importances,test_error,nonzero_features,train_error])

    if prob_type == 'Classification':
        final_tree_weights = cp.Variable(len(feature_ind), nonneg = True)
        loss = -cp.sum(cp.multiply(yTrain, train_pred_non_zero@ final_tree_weights) - cp.logistic(train_pred_non_zero @ final_tree_weights))
        objective_final =  (1/len(yTrain))*loss
        prob = cp.Problem(cp.Minimize(objective_final))
        prob.solve(solver = cp.MOSEK,mosek_params = {mosek.dparam.optimizer_max_time: 100.0})

        final_train_pred = train_pred_non_zero@final_tree_weights.value
        final_test_pred = test_pred_non_zero@final_tree_weights.value

        train_error =  sklearn.metrics.roc_auc_score(yTrain,final_train_pred)
        test_error = sklearn.metrics.roc_auc_score(yTest,final_test_pred)

        importances = np.sum(np.abs(test_pred_non_zero),axis = 0)
        feature_importances = pd.DataFrame(np.column_stack((features_sub,importances))\
                                          ,columns = ['Features','Importances'])
        feature_importances = pd.merge(all_features,feature_importances,on = 'Features',how = 'left').fillna(0).\
                                sort_values('Importances',ascending = False)
        nonzero_features = len(feature_ind)
        return([feature_importances,test_error,nonzero_features,train_error])
    
def sparse_ebm(xTrain,yTrain,xTest,yTest,lambd,threshold,prob_type):
    res = build_ebm(xTrain,yTrain,xTest,yTest,lambd,threshold,prob_type)
    return solve_lasso(xTrain,yTrain,xTest,yTest,lambd,threshold,prob_type,res)

def baseline(xTrain,yTrain,xTest,yTest,problem_type,nonzero):
    
    if problem_type == 'Regression':
        model = ExplainableBoostingRegressor(n_jobs = 3)
    if problem_type == 'Classification':
        model = ExplainableBoostingClassifier(n_jobs = 3)
    
    ebm = model.fit(xTrain,yTrain)
    ebm_global = ebm.explain_global()
    names = ebm_global._internal_obj.get('overall').get('names')
    scores = ebm_global._internal_obj.get('overall').get('scores')
    
    imp = pd.DataFrame(np.column_stack((names,scores)),columns = ['features','scores']).sort_values('scores',ascending = False)
    
    print(imp)
    acc =[]
    n_features = []
    se = []
    for i in nonzero:
        if i == 0:
            acc.append(0.5)
            se.append(0)
            n_features.append(i)
            continue
            
        to_use = imp.head(i)['features'].values
        trial = 0
        acc1 = []
        while trial < 4:
            
            if problem_type == 'Regression':
                model = ExplainableBoostingRegressor(n_jobs = 3)
            if problem_type == 'Classification':
                model = ExplainableBoostingClassifier(n_jobs = 3)
            
            ebm1 = model.fit(xTrain[to_use],yTrain)
            
            if problem_type == 'Regression':
                pred = ebm1.predict(xTest[to_use])
                acc1.append(np.sqrt(np.mean((yTest-pred)**2)))
                
            if problem_type == 'Classification':
                pred = ebm1.predict_proba(xTest[to_use])[:,1]
                acc1.append(sklearn.metrics.roc_auc_score(yTest,pred))
            trial = trial+1
        acc.append(np.mean(acc1))
        se.append(np.std(acc1))
        n_features.append(i)
    return acc,n_features,se



In [13]:
def load_breast_cancer():
    dataset = load_breast_cancer()
    X, y = dataset.data, dataset.target 
    X = pd.DataFrame(X,columns = dataset.feature_names)
    features = dataset.feature_names
    X = preprocessing.scale(X)
    X = pd.DataFrame(X,columns = features)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3)
    return xTrain, xTest, yTrain, yTest

def load_adult():
    data = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None)
    data.columns = [
            "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
            "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
            "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
        ]
    data['target'] = 0
    data['target'].loc[data['Income']== data['Income'].unique()[1]] = 1
    y = data['target']
    data.drop(['target','Occupation','Income'],axis = 1, inplace = True)
    data = pd.get_dummies(data, columns = ['WorkClass','Education','MaritalStatus','Relationship','Race','Gender'])
    data['NativeCountry'] = data['NativeCountry'] == ' United-States'
    data['NativeCountry'] = data['NativeCountry'].astype(int)
    features = data.columns
    X = preprocessing.scale(data)
    X = pd.DataFrame(X,columns = features)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3)
    return xTrain, xTest, yTrain, yTest

def load_heart():
    data = pd.read_csv('heart.csv')
    y = data['target']
    data.drop(['target'],axis = 1, inplace = True)
    features = data.columns
    X = preprocessing.scale(data)
    X = pd.DataFrame(X,columns = features)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3)
    return xTrain, xTest, yTrain, yTest

def load_credit():
    data = pd.read_csv('creditcard.csv')
    data = data.sample(10000)
    y = data['Class']
    data.drop(['Class'],axis = 1, inplace = True)
    features = data.columns
    X = preprocessing.scale(data)
    X = pd.DataFrame(X,columns = features)
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3)
    return xTrain, xTest, yTrain, yTest

def load_churn():
    data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
    data['target'] = data['Churn'].map({'No':0,'Yes':1})
    data = data[~data['target'].isnull()]
    data = data[data['TotalCharges']!= ' ']
    y = data['target']
    data = data.drop(['customerID','Churn','target'],axis = 1)
    data = pd.get_dummies(data, columns = ['gender','Partner', 'Dependents',
                                           'PhoneService','MultipleLines','InternetService','Contract',
                                           'PaymentMethod','OnlineSecurity','OnlineBackup','DeviceProtection'
                                          ,'TechSupport','StreamingTV','StreamingMovies','PaperlessBilling'])
    features = data.columns
    X = preprocessing.scale(data)
    X = pd.DataFrame(X,columns = features)
    for col in features:
        X[col] = X[col].fillna(np.nanmean(X[col]))
    xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3)
    return xTrain, xTest, yTrain, yTest

In [14]:
xTrain, xTest, yTrain, yTest = load_churn()

In [ ]:
ebm_test_acc = []
ebm_nonzero = []
ebm_train_acc = []
ntrials = 10
threshold = 10**-3
prob_type = 'Classification'
 #bisection function
LL = 0
RL = 1
to_find = 0
counter = 0
counter1 = 0
nfeatures = len(xTrain.columns)
count_array = []
trial = 0
lambd = 0
while trial <= ntrials:
    ebm_result = build_ebm(xTrain,yTrain,xTest,yTest,lambd,threshold,prob_type)
    while to_find <= nfeatures:
        lambd = (LL + RL)/2
        trial = 0
        result1 = []
        args = []
        results = solve_lasso(xTrain,yTrain,xTest,yTest,lambd,threshold,prob_type,ebm_result)
        test_acc = results[1]
        nonzero = results[2]
        train_acc = results[3]
        count_array.append(nonzero)
        freq = pd.DataFrame(np.column_stack(np.unique(count_array, return_counts = True)),columns = ['value','counts'])
        count_to_find = freq.loc[freq['value']==to_find]['counts'].values
        
        if count_to_find > 0 :
            counter1 = 0
            RL = lambd
            LL = 0
            to_find = to_find + 1

        elif counter1 >= 5:
            counter1 = 0
            RL = lambd
            LL = 0
            to_find = to_find + 1

        elif nonzero < to_find:
            RL = lambd
            counter1 = counter1 + 1 

        elif nonzero >= to_find:
            LL = lambd
            counter1 = counter1 + 1 
            
        print(freq,lambd)
        
        ebm_test_acc.append(test_acc)
        ebm_nonzero.append(nonzero)
        ebm_train_acc.append(train_acc)
    trial = trial + 1

   value  counts
0      0       1 0.5


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


   value  counts
0      0       2 0.25


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


   value  counts
0      0       3 0.125
   value  counts
0      0       3
1      1       1 0.0625
   value  counts
0      0       3
1      1       1
2      2       1 0.03125
   value  counts
0      0       3
1      1       1
2      2       1
3      3       1 0.015625


...

    value  counts
0       0       3
1       1       1
2       2       1
3       3       2
4       4       1
5       5       1
6       6       1
7       9       1
8      10       1
9      11      10
10     13       3
11     15       1
12     16       4
13     17       1
14     19       8
15     21       2
16     22      30
17     32      38 3.7945267214734967e-10


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


    value  counts
0       0       3
1       1       1
2       2       1
3       3       2
4       4       1
5       5       1
6       6       1
7       9       1
8      10       1
9      11      10
10     13       3
11     15       1
12     16       4
13     17       1
14     19       8
15     21       2
16     22      30
17     32      39 4.0655643444358893e-10


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


    value  counts
0       0       3
1       1       1
2       2       1
3       3       2
4       4       1
5       5       1
6       6       1
7       9       1
8      10       1
9      11      10
10     13       3
11     15       1
12     16       4
13     17       1
14     19       8
15     21       2
16     22      30
17     32      40 4.2010831559170856e-10


In [ ]:
base_line_acc,baseline_nonzero,baseline_se = baseline(xTrain,yTrain,xTest,yTest,'Classification',[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13])#np.unique(ebm_nonzero))

In [ ]:

plot_tradeoff_curve(ebm_test_acc,ebm_nonzero,'green',20,'lasso ebm')
plt.scatter(baseline_nonzero,base_line_acc,label = 'baseline',color = 'grey')
plt.errorbar(baseline_nonzero,base_line_acc, baseline_se,color = 'grey')

plt.legend()
plt.title('Churn')